In [ ]:
pip install tensorflow

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, TFBertForSequenceClassification, AutoTokenizer, AdamW
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel
import seaborn as sns
import re
import nltk
import string
import warnings
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder

import re
import string
import emoji
from bs4 import BeautifulSoup

pd.set_option('display.max_colwidth', 300)
warnings.filterwarnings("ignore")

In [ ]:
file_paths = ['/kaggle/input/goemotions-dataset/goemotions_1.csv', '/kaggle/input/goemotions-dataset2/goemotions_2.csv', '/kaggle/input/goemotions-dataset3/goemotions_3.csv']

merged_df = pd.concat(
    [
        pd.read_csv(file_path)
        for file_path in file_paths
    ],
    ignore_index=True,
)

merged_df.head()

In [ ]:
merged_df.shape

In [ ]:
df = merged_df.drop(
    ["id","author","subreddit","link_id","parent_id","created_utc","rater_id","example_very_unclear"],
    axis=1,
)
df = df.dropna(axis=0)
dict_keys = [key for key in df.columns if key != "text"]

df.head()

In [ ]:
df.head()

In [ ]:
cols = df.columns[df.columns.get_loc('anger'):df.columns.get_loc('neutral')+1]

# Veriyi eritme işlemi
df = df.melt(id_vars=['text'], value_vars=cols, value_name='emotion')

# Duygusal değeri 1 olan verileri seçin
df = df[df['emotion'] == 1]

# Duygu etiketlerini düzenleme
emotion_dict = {
    'annoyance': 'anger',
    'excitement': 'joy',
    'love': 'joy',
    'disappointment': 'sadness',
    'grief': 'sadness',
    'remorse': 'sadness',
    'embarrassment': 'sadness',
    'curiosity': 'surprise',
    'confusion': 'surprise'
}

df['variable'] = df['variable'].apply(lambda x: emotion_dict.get(x, x))

# Kullanılmayan sınıfları kaldırma
unused_classes = ['admiration', 'amusement', 'approval', 'caring', 'desire', 'disapproval', 'disgust', 'fear', 'gratitude', 'nervousness', 'optimism', 'pride', 'realization', 'relief']
df = df[~df['variable'].isin(unused_classes)]

# 'emotion' sütununu 'emotion_label' olarak yeniden adlandırma
df.rename(columns={'variable': 'emotion_label'}, inplace=True)

# Duygu gruplarını sayma
emotion_counts = df.groupby('emotion_label').size()

# Sonucu yazdırma
print(df.head(10))
print(emotion_counts)

In [ ]:
df.head()

In [ ]:
df['text'] = df['text'].str.lower()

In [ ]:
df

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}

In [ ]:
def clean_text(text):
    '''Clean emoji, Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = emoji.demojize(text)
    text = re.sub(r'\:(.*?)\:','',text)
    text = str(text).lower()    # Making Text Lowercase
    text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text

def clean_contractions(text, mapping):
    '''Clean contraction using contraction mapping'''    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    for word in mapping.keys():
        if ""+word+"" in text:
            text = text.replace(""+word+"", ""+mapping[word]+"")
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text

def clean_special_chars(text, punct, mapping):
    '''Cleans special characters present(if any)'''   
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

def correct_spelling(x, dic):
    '''Corrects common spelling errors'''   
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def remove_space(text):
    '''Removes awkward spaces'''   
    #Removes awkward spaces 
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
    text = clean_text(text)
    text = clean_contractions(text, contraction_mapping)
    text = clean_special_chars(text, punct, punct_mapping)
    text = correct_spelling(text, mispell_dict)
    text = remove_space(text)
    return text

In [ ]:
"""""
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}

def decontracted(df, column):
    def expand_contractions(phrase):
        for contraction, expansion in contraction_mapping.items():
            phrase = re.sub(r'\b' + re.escape(contraction) + r'\b', expansion, phrase)
        return phrase
    
    def preprocess_text(text):
        text = text.lower()  # Küçük harfe çevirme
        text = text.replace("'", "")  # Tek tırnak işaretlerini kaldırma
        text = re.sub('[^\w\s]', '', text)  # Noktalama işaretlerini kaldırma
        
        for punct, replacement in punct_mapping.items():
            text = text.replace(punct, replacement)
        
        for misspelled, correct in mispell_dict.items():
            text = text.replace(misspelled, correct)
        
        return text
    
    df[column] = df[column].apply(expand_contractions)
    df[column] = df[column].apply(preprocess_text)
    
    return df
"""""

In [ ]:
duplicates = df[df.duplicated(subset='text')]

In [ ]:
df.drop_duplicates(subset='text', keep='first', inplace=True)

In [ ]:
df["text"].duplicated().any()

In [ ]:
df = df.dropna()

In [ ]:
print(df['emotion_label'].nunique())

In [ ]:
emotion_counts = df.groupby('emotion_label').size()

plt.bar(emotion_counts.index, emotion_counts.values)
plt.xlabel('Emotion')
plt.ylabel('Count')
plt.title('Emotion Counts')
plt.xticks(rotation=45)
plt.show()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

text = ' '.join(df['text'])

# Kelime bulutunu oluşturma
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Emotion Word Cloud')
plt.show()

In [ ]:
word_frequencies = wordcloud.process_text(text)

for word, frequency in sorted(word_frequencies.items(), key=lambda x: x[1], reverse=True)[:100]:
    print(word, frequency) 

In [ ]:
import emoji

emoji_rows = df[df["text"].apply(lambda x: any(emoji.is_emoji(char) for char in x))]
emoji_rows = emoji_rows.head(20)
# Sonuçları yazdırma
print("Emojilerin olduğu satırlar:")
for index, row in emoji_rows.iterrows():
    print("Satır:", index)
    print("Metin:", row["text"])
    print("Duygu:", row["emotion_label"])
    print()

In [ ]:
def remove_emojis(df,col):
  emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
  
  df["text"] = df[col].str.replace(emoji_pattern, '',regex=True)

  return df

In [ ]:
df = remove_emojis(df, "text")


In [ ]:
df["text"]

In [ ]:
df["text"] = df["text"].apply(lambda x: re.sub(r'\[.*?\]', '', x))

In [ ]:
df["text"].head()

In [ ]:
matching_rows = df[df["text"].str.contains(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", regex=True)]

# Sonuçları yazdırma
print("Eşleşen satırlar:")
for index, row in matching_rows.iterrows():
    print("Satır:", index)
    print("Metin:", row["text"])
    print()

In [ ]:
has_numbers = df[df["text"].str.contains(r'\d', regex=True)]
has_numbers = has_numbers.head(20)
# Sonuçları yazdırma
print("Sayıları içeren satırlar:")
for index, row in has_numbers.iterrows():
    print("Satır:", index)
    print("Metin:", row["text"])
    print()

In [ ]:
df["text"] = df["text"].str.replace(r'\d+', '', regex=True)

In [ ]:
df["text"] = df["text"].replace(r'[^\w\s\'.;]', '', regex=True)

In [ ]:
df['text'].head(50)


In [ ]:
df['text'] = df['text'].apply(lambda x: ' '.join(word for word in x.split() if len(word) > 1))

In [ ]:
df['text']

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
!export CUDA_VISIBLE_DEVICES=0

In [ ]:
df.head()

In [ ]:
label_list = df['emotion_label'].unique()
label2id = {value: index for index, value in enumerate(label_list)}
label2id

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name)

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
encoded_data = tokenizer.batch_encode_plus(df['text'].tolist(), truncation=True, padding=True, return_attention_mask=True, return_tensors='tf')
labels = [label2id[label] for label in df["emotion_label"]]

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((dict(encoded_data), labels))

In [ ]:
df = df.drop("emotion", axis=1)

In [ ]:
df.head()

In [ ]:
X = df.drop(columns=["emotion_label"])
y = df["emotion_label"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

# Split the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, stratify=y_train)

In [ ]:
y_test.value_counts()


In [ ]:
y_val.value_counts()

In [ ]:
y.unique()

In [ ]:
train_labels, train_counts = np.unique(y_train, return_counts=True)
train_percentages = train_counts / len(y_train) * 100

# Calculate the percentage of each label in the test set
test_labels, test_counts = np.unique(y_test, return_counts=True)
test_percentages = test_counts / len(y_test) * 100

# Create a bar chart to compare the label distribution in the training and test sets
labels = y.unique()
x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, train_percentages, width, label='Training set')
rects2 = ax.bar(x + width/2, test_percentages, width, label='Test set')

ax.set_ylabel('Percentage')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# Add the percentage above each bar
for i, rect in enumerate(rects1):
    ax.text(rect.get_x() + rect.get_width() / 2.0, 1.05 * rect.get_height(),
            f'{train_percentages[i]:.2f}%', ha='center', va='bottom')
for i, rect in enumerate(rects2):
    ax.text(rect.get_x() + rect.get_width() / 2.0, 1.05 * rect.get_height(),
            f'{test_percentages[i]:.2f}%', ha='center', va='bottom')
plt.figure(figsize=(20,20))

plt.show()

In [ ]:
train_df = pd.DataFrame(X_train, columns=["text"])
train_df['emotion_label'] = y_train
train_df = train_df.sample(frac=1).reset_index(drop=True)


val_df = pd.DataFrame(X_val, columns=["text"])
val_df['emotion_label'] = y_val
val_df = val_df.sample(frac=1).reset_index(drop=True)


test_df = pd.DataFrame(X_test, columns=["text"])
test_df['emotion_label'] = y_test
test_df = test_df.sample(frac=1).reset_index(drop=True)

In [ ]:
train_encode = tokenizer.batch_encode_plus(train_df['text'].tolist(), truncation=True, padding=True, return_attention_mask=True, return_tensors='tf')
train_labels = [label2id[label] for label in train_df["emotion_label"]]

val_encode = tokenizer.batch_encode_plus(val_df['text'].tolist(), truncation=True, padding=True, return_attention_mask=True, return_tensors='tf')
val_labels = [label2id[label] for label in val_df["emotion_label"]]

test_encode = tokenizer.batch_encode_plus(test_df['text'].tolist(), truncation=True, padding=True, return_attention_mask=True, return_tensors='tf')
test_labels = [label2id[label] for label in test_df["emotion_label"]]


train_dataset =  tf.data.Dataset.from_tensor_slices((dict(train_encode), train_labels)).batch(32)
val_dataset =  tf.data.Dataset.from_tensor_slices((dict(val_encode), val_labels)).batch(32)
test_dataset =  tf.data.Dataset.from_tensor_slices((dict(test_encode), test_labels)).batch(32)

In [ ]:
num_classes = len(label2id)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=50)